Let's go for RNN based on https://github.com/AbhishekAnand18/ImageTextRecognition/blob/master/ImageTextRecognition_Code.ipynb

In [ ]:
# !pip install Pillow
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from IPython.display import clear_output
import time
import os
from skimage.filters import threshold_local
import keras
import random
from keras import backend as K
#from skimage.morphology import skeletonize
import itertools
import datetime


In [ ]:
dir_path = '/kaggle/input/orand-car-dataset/ORAND-CAR-2014/'
CAR_A_test_images_path = dir_path + 'CAR-A/a_test_images/'
CAR_A_train_images_path = dir_path + 'CAR-A/a_train_images/'
CAR_B_test_images_path = dir_path + 'CAR-B/b_test_images/'
CAR_B_train_images_path = dir_path + 'CAR-B/b_train_images/'

# def load_original_images():
#     images_path = []
#     for root, dirs, files in os.walk(dir_path):
#         for file in files:
#             if file.endswith('.png'):
#                 image_path = os.path.join(root, file)
#                 images_path.append(image_path.replace('\\','/'))
#     return images_path

# images_path = load_original_images()

def calculate_digit_count(label):
    return len(label)

def get_labels(image_dir,text_path):
  with open(text_path,'r') as f :
    lines = f.readlines()
  listt = []
  for line in lines :
    parts = line.strip().split("\t")
    listt.append([image_dir + parts[0],parts[1]])
  DF = pd.DataFrame(listt)
  DF = DF.rename(columns={0: 'image_path', 1: 'label'})
  return DF

def get_all_labels():
  CAR_A_test_text = dir_path + 'CAR-A/a_test_gt.txt'
  CAR_A_train_text = dir_path + 'CAR-A/a_train_gt.txt'
  CAR_B_test_text = dir_path + 'CAR-B/b_test_gt.txt'
  CAR_B_train_text = dir_path + 'CAR-B/b_train_gt.txt'
  a_test_label_df = get_labels(CAR_A_test_images_path,CAR_A_test_text)
  a_train_label_df = get_labels(CAR_A_train_images_path,CAR_A_train_text)
  b_test_label_df = get_labels(CAR_B_test_images_path,CAR_B_test_text)
  b_train_label_df = get_labels(CAR_B_train_images_path,CAR_B_train_text)
  all_labels = pd.concat([a_test_label_df , a_train_label_df , b_test_label_df , b_train_label_df],ignore_index=True)#.reset_index()
  return all_labels

def reset_data():
  all_labels = get_all_labels()
  print(len(all_labels))
  print(len(all_labels['image_path'].unique())) # no dupicate image name
  # all_labels["actual_digit_count"] = all_labels["label"].astype(str).apply(calculate_digit_count)
  return all_labels

all_labels = reset_data()

In [ ]:
all_labels

## Functionanl methods

In [ ]:
def get_size_dataFrame(all_labels,column):
    height_list = []
    width_list = []
    for image_path in all_labels[column]:
        height = Image.open(image_path).height
        width = Image.open(image_path).width

        height_list.append(height)
        width_list.append(width)

    sizes_df = pd.DataFrame({'width':width_list,'height':height_list})
    return sizes_df


def plot_random_image(image_label_df,from_path):
    random.seed(42)
    indices = list(range(len(image_label_df)))
    random.shuffle(indices)

    for idx in indices[:10]+[21,22]:
        target_image = image_label_df[from_path][idx]
        print("label:", image_label_df['label'][idx])
        print("target:", target_image)
        image = mpimg.imread(target_image)
        plt.imshow(image)
        plt.show()
        time.sleep(2)
        clear_output(wait=True)


def convert_to_grayscale(df, from_path,new_dir_path):
    try:
        os.makedirs(new_dir_path)
    except:
        pass

    for i, img_path in enumerate(df[from_path]):
        img = cv2.imread(img_path)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        save_path = os.path.join(new_dir_path, f"{i:05d}.png")
        cv2.imwrite(save_path, gray_img)

    df[new_dir_path] = [os.path.join(new_dir_path, f"{i:05d}.png") for i in range(len(df))]

    return df


def denoise_images(df, from_path, new_dir_path):
    try:
        os.makedirs(new_dir_path)
    except:
        pass

    for i, img_path in enumerate(df[from_path]):
        gray_img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        denoised_image = cv2.GaussianBlur(gray_img, (5, 5), 0)
        save_path = os.path.join(new_dir_path, f"{i:05d}.png")
        cv2.imwrite(save_path, denoised_image)

    df[new_dir_path] = [os.path.join(new_dir_path, f"{i:05d}.png") for i in range(len(df))]

    return df


def denoise_images_fastnlmeans(df, from_path, new_dir_path):
    try:
        os.makedirs(new_dir_path)
    except:
        pass

    for i, img_path in enumerate(df[from_path]):
        gray_img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        denoised_img = cv2.fastNlMeansDenoising(gray_img, None, h=10, templateWindowSize=7, searchWindowSize=21)
        save_path = os.path.join(new_dir_path, f"{i:05d}.png")
        cv2.imwrite(save_path, denoised_img)

    df[new_dir_path] = [os.path.join(new_dir_path, f"{i:05d}.png") for i in range(len(df))]

    return df


def resize_images_with_height(df,from_path, new_dir_path, target_height):
    try:
        os.makedirs(new_dir_path)
    except:
        pass

    for i, img_path in enumerate(df[from_path]):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        height, width = img.shape
        target_width = int(width * (target_height / height))
        resized_img = cv2.resize(img, (target_width, target_height))
        save_path = os.path.join(new_dir_path, f"{i:05d}.png")
        cv2.imwrite(save_path, resized_img)

    df[new_dir_path] = [os.path.join(new_dir_path, f"{i:05d}.png") for i in range(len(df))]
    return df


def resize_images(df,from_path, new_dir_path, target_height,target_width):
    try:
        os.makedirs(new_dir_path)
    except:
        pass

    for i, img_path in enumerate(df[from_path]):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        resized_img = cv2.resize(img, (target_width, target_height))
        save_path = os.path.join(new_dir_path, f"{i:05d}.png")
        cv2.imwrite(save_path, resized_img)

    df[new_dir_path] = [os.path.join(new_dir_path, f"{i:05d}.png") for i in range(len(df))]
    return df

def invert_images(df, from_path, new_dir_path):
    try:
        os.makedirs(new_dir_path)
    except:
        pass
    for i, img_path in enumerate(df[from_path]):
        bin_img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        inverted_img = cv2.bitwise_not(bin_img)
        save_path = os.path.join(new_dir_path, f"{i:05d}.png")
        cv2.imwrite(save_path, inverted_img)

    df[new_dir_path] = [os.path.join(new_dir_path, f"{i:05d}.png") for i in range(len(df))]
    return df

def binarize_images(df, from_path, new_dir_path):
    try:
        os.makedirs(new_dir_path)
    except:
        pass

    for i, img_path in enumerate(df[from_path]):
        im = Image.open(img_path)
        im = np.array(im)
        _, binary_im = cv2.threshold(im, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        binary_im = Image.fromarray(binary_im)
        save_path = os.path.join(new_dir_path, f"{i:05d}.png")
        binary_im.save(save_path)

    df[new_dir_path] = [os.path.join(new_dir_path, f"{i:05d}.png") for i in range(len(df))]
    return df

def denoise_images_fastnlmeans(df, from_path, new_dir_path):
    try:
        os.makedirs(new_dir_path)
    except:
        pass

    for i, img_path in enumerate(df[from_path]):
        gray_img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        denoised_img = cv2.fastNlMeansDenoising(gray_img, None, h=10, templateWindowSize=7, searchWindowSize=21)
        save_path = os.path.join(new_dir_path, f"{i:05d}.png")
        cv2.imwrite(save_path, denoised_img)

    df[new_dir_path] = [os.path.join(new_dir_path, f"{i:05d}.png") for i in range(len(df))]
    return df


def denoise_images(df, from_path, new_dir_path):
    try:
        os.makedirs(new_dir_path)
    except:
        pass

    for i, img_path in enumerate(df[from_path]):
        gray_img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        denoised_image = cv2.GaussianBlur(gray_img, (5, 5), 0)
        save_path = os.path.join(new_dir_path, f"{i:05d}.png")
        cv2.imwrite(save_path, denoised_image)

    df[new_dir_path] = [os.path.join(new_dir_path, f"{i:05d}.png") for i in range(len(df))]
    return df

def resize_with_padding(df, from_path, new_dir_path, target_height, target_width):
    try:
        os.makedirs(new_dir_path)
    except:
        pass

    for i, img_path in enumerate(df[from_path]):
        gray_img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        aspect_ratio = (gray_img.shape[1] / gray_img.shape[0])
        new_height = target_height
        new_width = int(aspect_ratio * new_height)

        resized_img = cv2.resize(gray_img, (new_width, new_height))
        if new_width < target_width:
            pad_width = target_width - new_width
            padded_img = cv2.copyMakeBorder(resized_img, 0, 0, 0, pad_width, cv2.BORDER_CONSTANT, value=255)
        else:
            padded_img = resized_img

        save_path = os.path.join(new_dir_path, f"{i:05d}.png")
        cv2.imwrite(save_path, padded_img)

    df[new_dir_path] = [os.path.join(new_dir_path, f"{i:05d}.png") for i in range(len(df))]
    return df

def local_threshold_images(df, from_path, new_dir_path, block_size, offset):
    try:
        os.makedirs(new_dir_path)
    except:
        pass
    for i, img_path in enumerate(df[from_path]):
        # Read the image in grayscale
        gray_img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # Perform local thresholding using the threshold_local method from skimage
        binary_img = threshold_local(gray_img, block_size, offset=offset, method='gaussian', mode='reflect', cval=0)

        # Convert the binary image to uint8 format (0 or 255)
        binary_img = (binary_img * 255).astype(np.uint8)

        # Save the binary image
        save_path = os.path.join(new_dir_path, f"{i:05d}.png")
        cv2.imwrite(save_path, binary_img)

    df[new_dir_path] = [os.path.join(new_dir_path, f"{i:05d}.png") for i in range(len(df))]
    return df



## Creating new images

In [ ]:
import cv2
import pandas as pd
import numpy as np
from skimage import io, transform, util
from skimage.transform import rotate
from skimage.util import random_noise

try:
    os.makedirs('/kaggle/working/created/')
except:
    pass

augmentation_params = {
    'resize_range': (0.7, 1.4),
    'noise_var': 0.004,
    'rotate_angle_range': (-10, 10)
}

new_data = {
    'image_path': [],
    'label': []
}

for i in range(3):
  for index, row in all_labels.iterrows():
      label = row['label']
      # Check if the label contains any digit except 0
      if '0' not in label:
          image_path = row['image_path']

          # Read the image
          img = io.imread(image_path, as_gray=True)

          # Resize by random ratio
          ratio = np.random.uniform(*augmentation_params['resize_range'])
          resized_img = transform.rescale(img, scale=ratio, mode='constant')

          # Rotate by a random angle
          angle = np.random.uniform(*augmentation_params['rotate_angle_range'])
          rotated_img = rotate(resized_img, angle, mode='edge')

          # Add Gaussian noise
          noisy_img = random_noise(rotated_img, var=augmentation_params['noise_var'])

          new_data['image_path'].append(noisy_img)
          new_data['label'].append(label)

new_df = pd.DataFrame(new_data)


In [ ]:
new_df

In [ ]:
last_image = len(all_labels)
new_im_paths = [os.path.join('/kaggle/working/created/', f"{i:05d}.png") for i in range(last_image, last_image + len(new_df))]
new_im_labels = new_df['label']
for img_array, new_im_path in zip(new_df['image_path'], new_im_paths):

    img_array = (img_array * 255).astype(np.uint8)

    # Ensure image array has the correct shape (height x width)
    if len(img_array.shape) == 3:
        img_array = img_array[:, :, 0]  # Extract the first channel (grayscale)

    # Save the grayscale image
    cv2.imwrite(new_im_path, img_array)
new_path_df = pd.DataFrame({'image_path': new_im_paths, 'label': new_im_labels})
all_labels = pd.concat([all_labels, new_path_df], ignore_index=True)



In [ ]:
all_labels

## Preproccesing

In [ ]:
# Gray scale
all_labels = convert_to_grayscale(all_labels,'image_path','/kaggle/working/grayscaled/')

In [ ]:
# sizes_df = get_size_dataFrame(all_labels,'/kaggle/working/grayscaled/')
# sizes_df.describe()

In [ ]:
# heights = sizes_df['height'].value_counts()
# value_counts_df = heights.reset_index()
# value_counts_df.columns = ['Value', 'Count']
# plt.bar(heights.index, heights.values)
# plt.xlabel('Unique Values')
# plt.ylabel('Count')
# plt.title('Count of Unique Values')
# plt.show()


In [ ]:
# widths = sizes_df['width'].value_counts()
# value_counts_df = widths.reset_index()
# value_counts_df.columns = ['Value', 'Count']
# plt.bar(widths.index, widths.values)
# plt.xlabel('Unique Values')
# plt.ylabel('Count')
# plt.title('Count of Unique Values')
# plt.show()


In [ ]:
# Denoising
all_labels = denoise_images_fastnlmeans(all_labels, '/kaggle/working/grayscaled/', '/kaggle/working/denoised_fastnlmeans/')
all_labels = denoise_images(all_labels, '/kaggle/working/denoised_fastnlmeans/', '/kaggle/working/denoised/')


# binarizing and inverting
# block_size = 53
# offset = 0
# all_labels = local_threshold_images(all_labels, '/kaggle/working/denoised_fastnlmeans/', '/kaggle/working/local/', block_size, offset)
all_labels = binarize_images(all_labels, '/kaggle/working/denoised/', '/kaggle/working/binarized/')
# all_labels = invert_images(all_labels, '/kaggle/working/denoised_fastnlmeans/', '/kaggle/working/inverted/')



In [ ]:
# resize to specific height
# target_height = 28
# all_labels = resize_images_with_height(all_labels,'./grayscaled/', './resized/', target_height)
all_labels = resize_with_padding(all_labels,'/kaggle/working/binarized/', '/kaggle/working/padded/',target_height=54,target_width=400)


In [ ]:
# all_labels = invert_images(all_labels, '/kaggle/working/inverted/', '/kaggle/working/inverted/')

In [ ]:
plot_random_image(all_labels,'/kaggle/working/padded/')

# Shuffling Data (Necessery)

In [ ]:
all_labels = all_labels.sample(frac=1, random_state=42)

## based on these two plots its better to resize images to 53*173

In [ ]:
digits= '0123456789'
img_h=54
img_w=400
#image Channels
img_c=1

# classes for softmax with number of letters +1 for blank space in ctc
num_classes=len(digits) +1
batch_size=64
max_length=8 # considering max length of ground truths labels to be 8

In [ ]:
def encode_numbers_labels(number):
    label_lst=[]
    for char in str(number):
        label_lst.append(digits.find(char)) # keeping 0 for blank and for padding labels
    return label_lst

In [ ]:
def numbers_from_labels(labels):
    txt=[]
    for ele in labels:
        if ele == len(digits): # CTC blank space
            txt.append("")
        else:
            #print(letters[ele])
            txt.append(digits[ele])
    return "".join(txt)

In [ ]:
def ctc_loss_function(args):
    y_pred, y_true, input_length, label_length = args
    y_pred = y_pred[:, 2:, :] # maybe need change based on my project and model architecture
    return K.ctc_batch_cost(y_true, y_pred, input_length, label_length)

In [ ]:
from keras.layers import Input, Conv2D, MaxPool2D, Dense,MaxPooling2D
from keras.layers import AveragePooling2D, Flatten, Activation, Bidirectional
from keras.layers import BatchNormalization, Dropout
from keras.layers import Concatenate, Add, Multiply, Lambda
from keras.layers import UpSampling2D, Reshape
from keras.layers import add, concatenate
from keras.layers import Reshape
from keras.models import Model
from keras.layers import LSTM,GRU
import tensorflow as tf

In [ ]:
def Image_digit_recogniser_model_1(stage,drop_out_rate=0.425):
    img_h=54
    img_w=400
    if K.image_data_format() == 'channels_first':
        input_shape = (1, img_w, img_h)
    else:
        input_shape = (img_w, img_h, 1)
    model_input=Input(shape=input_shape,name='img_input',dtype='float32')
    
    #input shape is (400, 54, 1)  
    #input shape is (400, 54, 64)  
    #input shape is (200, 27, 64)  
    #input shape is (200, 27, 128)
    #input shape is (100, 13, 128)
    #input shape is (100, 13, 256)
    #input shape is (100, 6, 512)
    #input shape is (100, 3, 512)
    #input shape is (100, 1536)
    
    # Convolution layer
    model = Conv2D(64, (3, 3), padding='same', name='conv1', kernel_initializer='he_normal')(model_input)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = MaxPooling2D(pool_size=(2, 2), name='max1')(model)

    model = Conv2D(128, (3, 3), padding='same', name='conv2', kernel_initializer='he_normal')(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = MaxPooling2D(pool_size=(2, 2), name='max2')(model)

    model = Conv2D(256, (3, 3), padding='same', name='conv3', kernel_initializer='he_normal')(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Conv2D(256, (3, 3), padding='same', name='conv4', kernel_initializer='he_normal')(model)
    model=Dropout(drop_out_rate)(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = MaxPooling2D(pool_size=(1, 3), name='max3')(model)

    model = Conv2D(512, (3, 3), padding='same', name='conv5', kernel_initializer='he_normal')(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Conv2D(512, (3, 3), padding='same', name='conv6')(model)
    model=Dropout(drop_out_rate)(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = MaxPooling2D(pool_size=(1, 2), name='max4')(model)

    model = Conv2D(512, (2, 2), padding='same', kernel_initializer='he_normal', name='con7')(model)
    model=Dropout(0.325)(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)

    # CNN to RNN
    model = Reshape(target_shape=(100, 1024), name='reshape')(model)

    model = Dense(96, activation='relu', kernel_initializer='he_normal', name='dense1')(model)

    # RNN layer (LTSM)
    model=Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='he_normal'), merge_mode='sum')(model)
    model=Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='he_normal'), merge_mode='concat')(model)

    # RNN layer (GRU)
    # model=Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='he_normal'), merge_mode='sum')(model)
    # model=Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='he_normal'), merge_mode='concat')(model)



    # transforms RNN output to character activations:
    model = Dense(num_classes, kernel_initializer='he_normal',name='dense2')(model)
    y_pred = Activation('softmax', name='softmax')(model)


    labels = Input(name='ground_truth_labels', shape=[max_length], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')

    #CTC loss function
    loss_out = Lambda(ctc_loss_function, output_shape=(1,),name='ctc')([y_pred, labels, input_length, label_length]) #(None, 1)

    if stage=='train':
        return model_input,y_pred,Model(inputs=[model_input, labels, input_length, label_length], outputs=loss_out)
    else:
        return Model(inputs=[model_input], outputs=y_pred)



In [ ]:
%pip install tensorflow-addons
import tensorflow_addons as tfa
radam=tfa.optimizers.RectifiedAdam()
from keras import optimizers
adam=optimizers.Adam()

In [ ]:
model_input,y_pred,img_digit_recog=Image_digit_recogniser_model_1('train')
test_func = K.function([model_input], [y_pred])
# img_digit_recog.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=adam)
img_digit_recog.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=radam)
img_digit_recog.summary()

# defining data generators

In [ ]:
class DataGenerator(keras.callbacks.Callback):
    def __init__(self, df, img_w, img_h, batch_size, max_number_len=8):
        self.img_h = img_h
        self.img_w = img_w
        self.batch_size = batch_size
        self.max_number_len = max_number_len

        self.df = df
        self.n = len(df)
        self.indexes = list(range(self.n))
        self.cur_index = 0
        self.imgs = np.zeros((self.n, self.img_h, self.img_w))
        self.numbers = df['label'].tolist()  # Extract related labels from DataFrame


    def build_data(self):
        print(self.n, " Image Loading start...")
        for i, img_file in enumerate(self.df['/kaggle/working/padded/']):
            img = cv2.imread(img_file)
            img = img[:,:,1]                               #Extracting Single Channel Image
            img = cv2.resize(img, (self.img_w, self.img_h))
            img = img /255
            self.imgs[i, :, :]= img
            if i%5000==0:
                print("Loaded Images: ",i)

        print("Number of Texts matches with Total Number of Images :",len(self.numbers) == self.n)
        print(self.n, " Image Loading finish...")

    def next_data(self):
        self.cur_index += 1
        if self.cur_index >= self.n:
            self.cur_index = 0
            random.shuffle(self.indexes)
        return self.imgs[self.indexes[self.cur_index]], self.numbers[self.indexes[self.cur_index]]

    def next_batch(self):
        while True:
            X_data = np.ones([self.batch_size, self.img_w, self.img_h, 1])
            Y_data = np.ones([self.batch_size, self.max_number_len])* -1
            input_length = np.ones((self.batch_size, 1)) * 40
            label_length = np.zeros((self.batch_size, 1))                   #label length for CTC
            source_str=[]                                                   #List to store Ground Truth Labels
            for i in range(self.batch_size):
                img, text = self.next_data() #getting the image and text data pointed by current index
                                    #taking transpose of image
                img=img.T
                img = np.expand_dims(img, -1)  #expanding image to have a single channel
                X_data[i] = img
                label=encode_numbers_labels(text) # encoding label text to integer list and storing in temp label variable
                lbl_len=len(label)
                Y_data[i,0:lbl_len] = label #Storing the label till its length and padding others
                label_length[i] = len(label)
                source_str.append(text) #storing Ground Truth Labels which will be accessed as reference for calculating metrics

        #Preparing the input for the Model
            inputs = {
                'img_input': X_data,
                'ground_truth_labels': Y_data,
                'input_length': input_length,
                'label_length': label_length,
                'source_str': source_str  # used for visualization only
            }
            #Preparing output for the Model and intializing to zeros
            outputs = {'ctc': np.zeros([self.batch_size])}
            yield (inputs, outputs) # Return the Prepared input and output to the Model

In [ ]:
test_set_precentage = int(len(all_labels) * 0.8)
val_set_precentage = int(len(all_labels) * 0.7 // 1)

In [ ]:
test_set_precentage,val_set_precentage

In [ ]:
train_gene = DataGenerator(all_labels[:val_set_precentage], img_w, img_h, batch_size, max_number_len=8)
train_gene.build_data()
train_num_batches=int(train_gene.n / batch_size)

In [ ]:
val_gen=DataGenerator(all_labels[val_set_precentage:test_set_precentage], img_w, img_h, batch_size, max_number_len=8)
val_gen.build_data()
val_num_batches=int(val_gen.n / batch_size)

In [ ]:
def decode_batch(test_func, number_batch):
    out = test_func([number_batch])[0] #returns the predicted output matrix of the model
    ret = []
    for j in range(out.shape[0]):
        out_best = list(np.argmax(out[j, 2:], 1))
        out_best = [k for k, g in itertools.groupby(out_best)]
        outstr = numbers_from_labels(out_best)
        ret.append(outstr)
    return ret

In [ ]:
def accuracies(actual_labels,predicted_labels,is_train):
    accuracy=0
    letter_acc=0
    letter_cnt=0
    count=0
    for i in range(len(actual_labels)):
        predicted_output=predicted_labels[i]
        actual_output=actual_labels[i]
        count+=1
        for j in range(min(len(predicted_output),len(actual_output))):
            if predicted_output[j]==actual_output[j]:
                letter_acc+=1
        letter_cnt+=max(len(predicted_output),len(actual_output))
        if actual_output==predicted_output:
            accuracy+=1
    final_accuracy=np.round((accuracy/len(actual_labels))*100,2)
    final_letter_acc=np.round((letter_acc/letter_cnt)*100,2)
    return final_accuracy,final_letter_acc

## callback visualizer

In [ ]:
class VizCallback(keras.callbacks.Callback):
    """
    The Custom Callback created for printing the Accuracy and Letter Accuracy Metrics at the End of Each Epoch
    """

    def __init__(self, test_func, text_img_gen,is_train,acc_compute_batches):
        self.test_func = test_func
        self.text_img_gen = text_img_gen
        self.is_train=is_train                #used to indicate whether the callback is called to for Train or Validation Data
        self.acc_batches=acc_compute_batches  # Number of Batches for which the metrics are computed typically equal to steps/epoch

    def show_accuracy_metrics(self,num_batches):
        """
        Calculates the accuracy and letter accuracy for each batch of inputs,
        and prints the avarage accuracy and letter accuracy across all the batches
        """
        accuracy=0
        letter_accuracy=0
        batches_cnt=num_batches
        while batches_cnt>0:
            number_batch = next(self.text_img_gen)[0]   #Gets the next batch from the Data generator
            decoded_res = decode_batch(self.test_func,number_batch['img_input'])
            actual_res=number_batch['source_str']
            acc,let_acc=accuracies(actual_res,decoded_res,self.is_train)
            accuracy+=acc
            letter_accuracy+=let_acc
            batches_cnt-=1
        accuracy=accuracy/num_batches
        letter_accuracy=letter_accuracy/num_batches
        if self.is_train:
            print("\nTrain Average Accuracy of "+str(num_batches)+" Batches: ",np.round(accuracy,2)," %")
            print("Train Average Letter Accuracy of "+str(num_batches)+" Batches: ",np.round(letter_accuracy,2)," %")
        else:
            print("Validation Average Accuracy of "+str(num_batches)+" Batches: ",np.round(accuracy,2)," %")
            print("Validation Average Letter Accuracy of "+str(num_batches)+" Batches: ",np.round(letter_accuracy,2)," %")


    def on_epoch_end(self, epoch, logs={}):
        self.show_accuracy_metrics(self.acc_batches)

In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
early_stop=EarlyStopping(monitor='val_loss',patience=2,restore_best_weights=True)
model_chk_pt=ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', save_best_only=False,save_weights_only=True,verbose=0, mode='auto', period=2)
logdir = os.path.join("logs_127", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs_127
clear_output()

In [ ]:
viz_cb_train = VizCallback( test_func, train_gene.next_batch(),True,train_num_batches)
viz_cb_val = VizCallback( test_func, val_gen.next_batch(),False,val_num_batches)

In [ ]:
new_df = None
sizes_df = None
heights = None
widths = None
# all_labels.drop(['./grayscaled/'])
# remove_directory('/content/grayscaled/')
# remove_directory('/content/denoised/')
# remove_directory('/content/created/')

In [ ]:
history = img_digit_recog.fit(
    train_gene.next_batch(),
    steps_per_epoch=int(train_gene.n / batch_size),
    epochs=30,
    callbacks=[viz_cb_train, viz_cb_val, train_gene, val_gen, tensorboard_callback, early_stop, model_chk_pt],
    validation_data=val_gen.next_batch(),
    validation_steps=int(val_gen.n / batch_size)
)
img_digit_recog.save("model_16.h5")
print("model saved")


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()



## now measure our model on test dataset

actually we dont have dataset

so we can use new data or train out model on smaller dataset

In [ ]:
def decode_label(out):
    out_best = list(np.argmax(out[0,2:], axis=1))

    out_best = [k for k, g in itertools.groupby(out_best)]  # remove overlap value

    outstr=numbers_from_labels(out_best)
    return outstr

In [ ]:
model=Image_digit_recogniser_model_1('predict')
# model = keras.models.load_model('first_model.h5',compile=False)
model.load_weights('model_16.h5')

In [ ]:
test_data = all_labels[test_set_precentage:]
val_img_names=test_data['/kaggle/working/padded/'].values
val_labels=test_data['label'].values

In [ ]:
def single_image_resizer_with_padding(image_array, target_height, target_width):
    gray_img = image_array
    aspect_ratio = (gray_img.shape[1] / gray_img.shape[0]) * 1.1
    new_height = target_height
    new_width = int(aspect_ratio * new_height)

    resized_img = cv2.resize(gray_img, (new_width, new_height))
    if new_width < target_width:
        pad_width = target_width - new_width
        padded_img = cv2.copyMakeBorder(resized_img, 0, 0, 0, pad_width, cv2.BORDER_CONSTANT, value=255)
    else:
        padded_img =  cv2.resize(resized_img, (target_width, target_height))

    return padded_img

In [ ]:
def test_data_output_Prediction(model, test_img_names, test_labels):
    num_classes = 10  # Assuming there are 10 digits (0 to 9)
    accuracy = 0
    letter_acc = [0] * num_classes
    letter_total = [0] * num_classes
    count = 0
    all_predicted = []
    letter_mis_match = []

    for i in range(len(test_labels)):
        test_img = cv2.imread(test_img_names[i])
        test_image = single_image_resizer_with_padding(test_img,54,400)
        # print("1 :",test_image.shape)
        test_image = test_image[:, :, 1]
        # print("2 :",test_image.shape)
        test_image = test_image.T
        # print("3 :",test_image.shape)
        test_image = np.expand_dims(test_image, axis=-1)
        # print("4 :",test_image.shape)
        test_image = np.expand_dims(test_image, axis=0)
        # print("5 :",test_image.shape)
        test_image = test_image / 255
        model_output = model.predict(test_image, verbose=-1)
        predicted_output = decode_label(model_output)
        # print("output :",predicted_output)
        actual_output = test_labels[i]
        all_predicted.append([test_img_names[i], predicted_output])
        count += 1
        mis_match = 0

        for j in range(min(len(predicted_output), len(actual_output))):
            if predicted_output[j] == actual_output[j]:
                letter_acc[int(predicted_output[j])] += 1
            else:
                mis_match += 1
            letter_total[int(actual_output[j])] += 1

        letter_mis_match.append(mis_match)
        if actual_output == predicted_output:
            accuracy += 1

        if (count % 400) == 0:
            print("Processed", count, "Images")

    digit_accuracy = [acc / total if total > 0 else 0 for acc, total in zip(letter_acc, letter_total)]
    return accuracy, letter_acc, letter_total, digit_accuracy, letter_mis_match, all_predicted

In [ ]:
synth_val_accuracy,letter_acc,synth_val_letter_acc,synth_val_letter_cnt,synth_val_mis_match,all_predicted=test_data_output_Prediction(model,val_img_names,val_labels)
synth_val_accuracy,letter_acc,synth_val_letter_acc,synth_val_letter_cnt#,synth_val_mis_match


In [ ]:
true_list = []
for i in range(len(all_labels['label'].iloc[test_set_precentage:])):
  if str(all_labels['label'].iloc[test_set_precentage + i]) == str(all_predicted[i][1]):
    true_list.append(1)
  else:
    print(all_labels['label'].astype(str).iloc[test_set_precentage + i]," == ",all_predicted[i][1],"image -->",all_predicted[i][0])
    true_list.append(0)
trues = pd.DataFrame(true_list)
trues[0].describe()

In [ ]:
def remove_last_digit_if_one(predicted_outputs):
    cleaned_outputs = []
    for output in predicted_outputs:
        #if output[1][-1] == '2' and  output[1][-2] == '9':
        if output[1][-1] == '1':
             #cleaned_outputs.append([output[0],output[1][:-2]])
            cleaned_outputs.append([output[0],output[1][:-1]])
        else:
            cleaned_outputs.append([output[0],output[1]])
    return cleaned_outputs

all_predicted = remove_last_digit_if_one(all_predicted)

As can be seen, due to the large number of zeros in the training dataset, most of the prediction errors have been replaced by zeros

In [ ]:
pre_digits_df = pd.DataFrame(letter_acc)
pre_digits_df.append({0:0},ignore_index=True)


all_digits = []
for i in all_labels['label'].iloc[test_set_precentage:]:
  for ii in i:
    all_digits.append(ii)
print(len(all_digits))
digits_df = pd.DataFrame(all_digits)
digits_df.append({0:0},ignore_index=True)
digit_counts = digits_df[0]

digit_counts = pd.DataFrame(digit_counts)
digit_counts = digit_counts[0].value_counts()
digit_counts = pd.DataFrame(digit_counts)


df1 = digit_counts.sort_index()
df2 = pre_digits_df.sort_index()

# Create a figure and axis
fig, ax = plt.subplots()

# Plot the data for the first dataframe
ax.bar(df1.index, df1[0], width=0.4, align='center', label='actual')

# Plot the data for the second dataframe (shifted by 0.4 for side-by-side comparison)
ax.bar(df2.index + 0.4, df2[0], width=0.4, align='center', label='predicted')

# Set labels and title
ax.set_xlabel('Digit')
ax.set_ylabel('Count')
ax.set_title('Comparison of Digit Counts')
ax.set_xticks(df1.index)
ax.legend()

# Show the plot
plt.show()


## Exploring all digits count in our main dataset

In [ ]:
all_digits = []
for i in all_labels['label']:
  for ii in i:
    all_digits.append(ii)
print(len(all_digits))
digits_df = pd.DataFrame(all_digits)

In [ ]:
import matplotlib.pyplot as plt

# Calculate the frequency of each digit
digit_counts = digits_df[0].value_counts()

# Plot the count of each digit
plt.figure(figsize=(10, 6))
digit_counts.plot(kind='bar')
plt.title('Count of Each Digit')
plt.xlabel('Digit')
plt.ylabel('Count')
plt.show()


## load DIDA into dataframe

In [ ]:
dida_images_dir_path = '/kaggle/input/DIDA-for-test/DIDA_1/'
import pandas as pd
dida_labels_dir_path = '/kaggle/input/dida-labels/DIDA_12000_String_Digit_Labels.csv'
your_header = ['image_name', 'label']
dida_labels = pd.read_csv(dida_labels_dir_path)
dida_labels.columns = your_header


dida_labels['image_name'] = dida_labels['image_name'].astype(str)
dida_labels['label'] = dida_labels['label'].astype(str)
dida_paths = []
for row in dida_labels['image_name'].astype(str) :
    full_path = dida_images_dir_path + row + '.jpg'
    dida_paths.append(full_path)
dida_labels['image_path'] = dida_paths
dida_labels.drop(columns=['image_name'], inplace=True)
dida_labels

dida_sample = dida_labels.sample(n=500, random_state=42)
dida_sample

In [ ]:
# image = cv2.imread(dida_images_dir_path + '362.jpg')
# image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# plt.imshow(image_rgb)
# plt.axis('off')  # Turn off axis labels
# plt.show()

In [ ]:
dida_sample

In [ ]:
def preprocess_and_save_images(df, from_path, new_dir_path):
    try:
        os.makedirs(new_dir_path)
    except:
        pass

    for i, path in enumerate(df[from_path]):
        im_g = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  
        im_g = cv2.fastNlMeansDenoising(im_g, None, 10, 7, 21)
        im_g = cv2.GaussianBlur(im_g, (5, 5), 0) 
        _, im_g = cv2.threshold(im_g, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)  
        target_height = 54
        target_width = 400 
        aspect_ratio = (im_g.shape[1] / im_g.shape[0])
        new_height = target_height
        new_width = int(aspect_ratio * new_height)
        resized_im_g = cv2.resize(im_g,(new_width, new_height))
        if new_width < target_width:
            pad_width = target_width - new_width
            padded_im_g = cv2.copyMakeBorder(resized_im_g, 0, 0, 0, pad_width, cv2.BORDER_CONSTANT, value=255)
        else:
            padded_im_g = resized_img

        save_path = os.path.join(new_dir_path, f"{i:05d}.png")
        cv2.imwrite(save_path, padded_im_g)

    df[new_dir_path] = [os.path.join(new_dir_path, f"{i:05d}.png") for i in range(len(df))]
    return df


dida_sample = preprocess_and_save_images(dida_sample, 'image_path','dida')


In [ ]:
plot_random_image(dida_sample,'don')

In [ ]:
test_data = dida_sample
dida_img_names=test_data['don'].values
dida_labels=test_data['label'].values

In [ ]:
synth_val_accuracy,letter_acc,synth_val_letter_acc,synth_val_letter_cnt,synth_val_mis_match,all_predicted=test_data_output_Prediction(model,dida_img_names,dida_labels)
synth_val_accuracy,letter_acc,synth_val_letter_acc,synth_val_letter_cnt#,synth_val_mis_match


In [ ]:
true_list = []
for i in range(len(dida_sample)):
  if str(dida_sample['label'].iloc[i]) == str(all_predicted[i][1]):
    true_list.append(1)
    print(dida_sample['label'].astype(str).iloc[i]," == ",all_predicted[i][1],"image -->",all_predicted[i][0])
  else:
    true_list.append(0)
trues = pd.DataFrame(true_list)
trues[0].describe()

In [ ]:
all_predicted

In [ ]:
image = mpimg.imread('/kaggle/input/DIDA-for-test/DIDA_1/5405.jpg')
print("image : ",image)
plt.imshow(image)
plt.show()